# Workbook for accessing loc.gov API

This workbook provides prompts for working with the loc.gov API. 
For reference, also see: 

* Laura Wrubel's [documentation of the API](https://libraryofcongress.github.io/data-exploration/)
* New Library [explanation of the API](https://www.loc.gov/apis/json-and-yaml/).
* Library's [Data Exploration notebooks series](https://github.com/LibraryOfCongress/data-exploration).

## Setup
More advanced work may require additional modules, but the 
basic actions of interacting with an API can be acomplished 
using the requests module. 
Related work may benefit from Python's JSON and CSV libraries, 
as well as the os, os.path, and glob libraries
to support working with local files.     

In [5]:
# write your import statements
import requests

## Making an API call

Use requests to scrape some data from an API endpoint. In this case, we can use the Library of Congress search function, which is a REST API that responds to HTTP requests.

The documentation for requests can be found here: http://docs.python-requests.org/en/master/

There are multiple loc endpoints, which access different collection aspects. 
Consult the documentation here: https://www.loc.gov/apis/json-and-yaml/requests/. 

The endpoint for the search query is http://www.loc.gov/search/

The response can be modified by supplying various parameters:
* To request the json format, use the `fo=json` parameter. 
* To provide a search query, use the `q` parameter, for example how would you search for images of kittens? 
* To specify images, use the `fa=online-format:image` paramter.

In [3]:
searchendpoint = 'http://www.loc.gov/search/'

parameters = {
    'fo' : 'json',
    'q'  : 'kittens',
    'fa' : 'online-format:image'
}

In [6]:
r = requests.get(searchendpoint, params=parameters)

In [9]:
r.url

'https://www.loc.gov/search/?fo=json&q=kittens&fa=online-format%3Aimage'

You can explore the headers by calling `r.headers`, which is a dictionary. You can see the items in the dictionary using the `keys()` function:

In [12]:
for key in r.headers.keys():
    print(key)

Date
Content-Type
Content-Length
Connection
X-Robots-Tag
X-Frame-Options
Access-Control-Allow-Origin
Referrer-Policy
Strict-Transport-Security
X-Content-Type-Options
ETag
Expires
Content-Security-Policy
X-Grace
X-Nearside-Cache
X-Nearside-Cache-Hits
Cache-Control
Age
Accept-Ranges
CF-Cache-Status
Server
CF-RAY


The response is a JSON object. You can use the JSON library to parse this. 
You can also use requests' built-in json function:  

In [14]:
r.json().keys()

dict_keys(['breadcrumbs', 'expert_resources', 'facet_trail', 'facet_views', 'facets', 'form_facets', 'options', 'pagination', 'results', 'search', 'timestamp', 'views'])

### What is the pagination info?

In [22]:
r.json()['pagination'].keys()

dict_keys(['current', 'first', 'from', 'last', 'next', 'of', 'page_list', 'perpage', 'perpage_options', 'previous', 'results', 'to', 'total'])

In [19]:
r.json()['pagination']['total']

1257

In theory, you could use this as the definition of a range function and request the list of all 31,000+ kitten images. 

### Where are the data about the items? 

Hint: what are the `results`?

In [27]:
len(r.json()['results'])

25

This makes sense since the pagination is set to 25 per page, so this is likely the items. In other words, the things we want to take a look at. 

In [26]:
r.json()['results'][0].keys()

dict_keys(['access_restricted', 'aka', 'campaigns', 'contributor', 'date', 'dates', 'description', 'digitized', 'extract_timestamp', 'group', 'hassegments', 'id', 'image_url', 'index', 'item', 'language', 'location', 'location_country', 'mime_type', 'number', 'number_former_id', 'online_format', 'original_format', 'other_title', 'partof', 'related', 'reproductions', 'resources', 'shelf_id', 'site', 'subject', 'timestamp', 'title', 'unrestricted', 'url'])

### How can we find the URL to the images?

Start by isolating the item information. . . 

In [29]:
kittenList = r.json()['results']

print(len(kittenList))

25


In [31]:
kittenList[0].keys()

dict_keys(['access_restricted', 'aka', 'campaigns', 'contributor', 'date', 'dates', 'description', 'digitized', 'extract_timestamp', 'group', 'hassegments', 'id', 'image_url', 'index', 'item', 'language', 'location', 'location_country', 'mime_type', 'number', 'number_former_id', 'online_format', 'original_format', 'other_title', 'partof', 'related', 'reproductions', 'resources', 'shelf_id', 'site', 'subject', 'timestamp', 'title', 'unrestricted', 'url'])

What is the `url` value?

In [32]:
for kitten in kittenList: 
    print(kitten['url'])

https://www.loc.gov/item/2016892679/
https://www.loc.gov/item/20002503/
https://www.loc.gov/item/sm1880.04831/
https://www.loc.gov/item/2013646722/
https://www.loc.gov/item/2017650796/
https://www.loc.gov/item/2008461635/
https://www.loc.gov/item/20020550/
https://www.loc.gov/item/04028966/
https://www.loc.gov/item/jukebox-668708/
https://www.loc.gov/item/sm1883.19433/
https://www.loc.gov/item/sm1879.15529/
https://www.loc.gov/item/2016780779/
https://www.loc.gov/item/2016770792/
https://www.loc.gov/item/2016816441/
https://www.loc.gov/item/2016817090/
https://www.loc.gov/item/2008660988/
https://www.loc.gov/item/2005681032/
https://www.loc.gov/item/2016796464/
https://www.loc.gov/item/2022653071/
https://www.loc.gov/item/2022652300/
https://www.loc.gov/item/2014717546/
https://www.loc.gov/item/2002706499/
https://www.loc.gov/item/22017222/
https://www.loc.gov/item/sm1882.12947/
https://www.loc.gov/item/2002697129/


### Can you save the item metadata to a local file?